#### Setup Environment

In [13]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Administering Your GIS Organizations Using ArcGIS API for Python

## Overview

<img src="./img/api_for_python.png" width=150 />

- The ArcGIS ecosystem is vast
- Organizations can have multiple versions of any product or multiple products to manage
- How do you manage this?

## The Way of the Python

<img class=".imgA1" src="./img/waysofscience.jpg" width=175 />

- The Python API allows administrators to manage, update and control what happens on your server
- Script from your favorite IDE or Notebook environment
- Cross platform support

## What Can We Manage?

<table  style='font-family:"Courier New", Courier, monospace; font-size:180%' width=50%>
  
  <tr>
    <td>Users</td>
    <td><img src="./img/users.png", width=60 /></td>
  </tr>
  <tr>
    <td>Content</td>
    <td><img src="./img/content.png", width=60 /></td>
  </tr>
  <tr>
    <td>Organization Policies/Licensing</td>
    <td><img src="./img/infrastructure.png", width=60 /></td>
  </tr>
  <tr>
    <td>Groups</td>
    <td><img src="./img/groups.png" width=60 /></td>
  </tr>
 
</table>

## Getting Started

## Understand the `GIS` Object

The `GIS` object is the way users connect to ArcGIS Online and/or Enterprise

- It doesn't matter if you are an administrator or a user, we must start here.

#### Connecting to your `GIS` 

The ArcGIS API for Python support multiple ways of connecting to your Web GIS deployment, whether it be ArcGIS Online, ArcGIS Enterprise or ArcGIS Enterprise for Kubernetes.

##### Anonymously

In [14]:
import warnings
import pandas as pd

from arcgis.gis import GIS

warnings.filterwarnings("ignore", category=UserWarning)
gis_anon = GIS() #anonymous connection

In [15]:
gis_anon.content.search("rivers")

[<Item title:"Natural Environment Database Map in Taiwan" type:Web Mapping Application owner:edchangtw>,
 <Item title:"Green Infrastructure in the Willamette River Watershed" type:StoryMap owner:crecor_pdxedu>,
 <Item title:"Bureau of Labor Statistics Monthly Unemployment (latest 14 months)" type:Feature Layer Collection owner:esri_demographics>,
 <Item title:"Sentinel-2 10m Land Use/Land Cover Time Series" type:Imagery Layer owner:esri_imagery>,
 <Item title:"Housing with Mortgages" type:Web Map owner:AtlasPublisher>,
 <Item title:"Mardi Gras Pass - A new geographic feature on the coast of Louisiana" type:Web Mapping Application owner:LDOTG1308>,
 <Item title:"Suggested Sites for LA River Parks" type:Web Mapping Application owner:fraanky3>,
 <Item title:"Renmark Storymap from South Australia" type:Web Mapping Application owner:Ritgisonline>,
 <Item title:"Active Hurricanes, Cyclones and Typhoons" type:Feature Layer Collection owner:esri_livefeeds2>,
 <Item title:"National Wild and Sce

##### Built-In

- username/password login method
- usersname are case sensitive 

```python
gis = GIS(username='fakeaccount', password='fakepassword')
gis = GIS(url="https://www.mysite.com/portal", username='fakeaccount', password='fakepassword')
```

**Protecting Built-In Credentials**

- using `profiles` will help protect username and passwords.  
- prevents accidental sharing

1. Create a `GIS` object with the extra `profile` parameter

```python
gis = GIS(url="https://www.mysite.com/portal", 
          username='fakeaccount', 
          password='fakepassword', 
          profile='portal_profile')
```

2. Now connect using the `profile`

```python
gis = GIS(profile='portal_profile')
```

**What Happened?**

Instead of keeping your password in plain text, now we leverage the operating system's credential store for the logged in user.  The credentials never get passed on when you use profiles.

##### Other Login Methods

- LDAP
- IWA
- PKI
- OAuth 2.0.
- API Key

## User Management

<img src="./img/usermanagement.jpg" width=90 />


Users fuel your system. As an administrator your job is to ensure they can put up there content and know the site is reliable and safe.  The Python API is a tool to do just that!

### Connecting as an Administrator

In [16]:
gis = GIS(
    profile='your_online_admin_profile', 
    verify_cert=False
)

Setting `verify_cert` to False is a security risk, use at your own risk.


In [17]:
if hasattr(gis, "admin"):
    print("You are connected as: Administrator.")
else:
    print("You are connected as: Non-Administrator.")

You are connected as: Administrator.


In [18]:
# Check admin property by typing . then Tab key
gis.

GIS @ https://geosaurus.maps.arcgis.com version:2024.2

### Working with Existing Users

The UserManager class in the API provides capabilites for controlling members of the organization. An object of the class can be directly initialized from the _users_ property on the GIS object.

In [19]:
umgr = gis.users
umgr

< UserManager at https://geosaurus.maps.arcgis.com >

#### Search for Users

In [20]:
users = umgr.search("username:a*")
users

[<User username:achapkowski_geosaurus>,
 <User username:achapkowski_geosaurus3>,
 <User username:amani_geosaurus>,
 <User username:amolina2023>,
 <User username:andrew57>,
 <User username:api_data_owner>,
 <User username:arcgis_python>,
 <User username:arcgislearn_geosaurus>,
 <User username:ArcGISPyAPIBot>]

#### List User's Groups

The _user_groups_ method is designed for reporting the the Group id value for each group a user is a member of, providing a management tool to administrators for handling group membership within the organization.

In [21]:
umgr.user_groups?

Signature:
umgr.user_groups(
    users: 'Union[list[str], list[User]]',
    max_results: 'int' = -1,
)
Docstring:
Givens a List of Users, the ``user_groups`` method will report back all group ids
that each :class:`~arcgis.gis.User` belongs to. ``user_groups`` is designed to be a reporting
tool for administrators so they can easily manage a user or users groups.

================  ========================================================
**Parameter**      **Description**
----------------  --------------------------------------------------------
users             Required List. An array of User objects or usernames.
----------------  --------------------------------------------------------
max_results       Optional Integer. A limiter on the number of groups
                  returned for each user.
================  ========================================================

:return:
    List of dictionaries with each :class:`~arcgis.gis.User` object's group ids.

.. code-block:: python



In [22]:
for grpinfo in umgr.user_groups(users):
    print(f"{grpinfo['username']} is a member of {len(grpinfo['groups'])} group(s)\n{'-' * 25}")
    for grp in grpinfo['groups']:
        print(f"{' ' * 3}{grp['title']} -> owned by {gis.groups.get(grp['id']).owner}")
    print("\n")

amani_geosaurus is a member of 5 group(s)
-------------------------
   Properties at risk -> owned by amani_geosaurus
   2017 all hands -> owned by amani_geosaurus
   Land_subsidence -> owned by amani_geosaurus
   Demo Initiative1 Initiative Collaboration Group -> owned by amani_geosaurus
   grp_2e31a1 -> owned by andrew57


andrew57 is a member of 5 group(s)
-------------------------
   test group 6abe -> owned by andrew57
   test group fc3d -> owned by andrew57
   test group bf19 -> owned by andrew57
   test group 1fed -> owned by andrew57
   sdfgsdfgsdfh Content -> owned by andrew57


arcgislearn_geosaurus is a member of 1 group(s)
-------------------------
   OSM Iceland Data -> owned by andrew57


achapkowski_geosaurus is a member of 1 group(s)
-------------------------
   OSM Iceland Data -> owned by andrew57


api_data_owner is a member of 5 group(s)
-------------------------
   ArcGIS API for Python "Try it Live" Samples -> owned by api_data_owner
   grp_006bda -> owned by api_

#### List a User's Items

The _items()_ method on the _User_ class returns a list of items owned by the user, providing a _folder_ parameter to inspect each folder a user has created as well.

In [23]:
u0 = users[0]
uprops = [d for d in dir(u0) if not d.startswith("_")]
for a,b,c,d,e in zip(uprops[1::5], uprops[2::5], uprops[3::5], uprops[4::5], uprops[5::5]):
    print(f"{a:30}{b:30}{c:30}{d:30}{e:30}")
print(f"{uprops[80]:30}")

assignedCredits               availableCredits              bundles                       categories                    clear                         
copy                          created                       culture                       cultureFormat                 delete                        
delete_thumbnail              description                   disable                       disabled                      download_thumbnail            
email                         emailStatus                   enable                        esri_access                   expire_password               
favGroupId                    firstName                     folders                       fromkeys                      fullName                      
generate_direct_access_url    get                           get_thumbnail                 get_thumbnail_link            groups                        
homepage                      id                            idpUsername                   invi

In [24]:
for u in users[:4]:
    print(f"{u.username.replace('geosaurus','wdp'):27}\n{'-' * 20}")
    for fldr in gis.content.folders.list(owner=u):
        if fldr.properties.get("name", None):
            print(f"{' ' * 8}{fldr.properties['name']:50}{len(list(fldr.list()))}")
        else:
            print(f"{' ' * 8}{fldr.properties['title']:50}{len(list(fldr.list()))}")
    print("\n")

achapkowski_wdp            
--------------------
        Root Folder                                       21


achapkowski_wdp3           
--------------------
        Root Folder                                       0


amani_wdp                  
--------------------
        Root Folder                                       2


amolina2023                
--------------------
        Root Folder                                       0




#### Update User Properties

In [25]:
user = gis.users.get("arcgislearn_geosaurus")
user

<User username:arcgislearn_geosaurus>

In [26]:
user.thumbnail

'dino.png'

In [27]:
user.update(
    thumbnail=r"./img/dino.png"
)

True

In [28]:
user.thumbnail

'dino.png'

### Creating New Users

#### Create user with New Member Default role and user type

An organization can set [New Member Defaults](https://doc.arcgis.com/en/arcgis-online/administer/configure-new-member-defaults.htm). Default administrators and those with the appropriate privileges can specify the user type, member role, number of credits, groups, and other member properties to assign by default when adding or inviting new members to the organization. Let's check whether any are set on our organization:

In [29]:
gis.users.user_settings

{'role': 'org_publisher',
 'userLicenseType': 'creatorUT',
 'groups': [],
 'userType': 'arcgisonly',
 'apps': [],
 'appBundles': [],
 'categories': []}

In [30]:
import uuid
username = f"RUser{uuid.uuid4().hex[:4]}"
password = f"!{uuid.uuid4().hex[:8]}A"

In [31]:
new_user = umgr.create(
    username=username, 
    password=password, 
    firstname="Demon",
    lastname="Stration",
    email='testsadf@esri.com'
)

new_user

<User username:RUser29c0>

In [32]:
print(f"{new_user.username:12}{new_user.role:15}{new_user.userLicenseTypeId}")

RUser29c0   org_publisher  creatorUT


#### Handling Security and Password Resets

<img src="./img/password-reset.jpg" />

In [33]:
new_password = f"!{uuid.uuid4().hex[:8]}A"

In [34]:
new_user.reset(
    password=password,
    new_password=new_password,
    new_security_question=1,
    new_security_answer=uuid.uuid4().hex[:10],
    reset_by_email=False,
)

True

In [35]:
GIS(username=username, 
    password=new_password, 
    verify_cert=False).users.me

Setting `verify_cert` to False is a security risk, use at your own risk.


<User username:RUser29c0>

#### Deleting the User

- user must not own items
- user must not have licenses checked out

See The _[Deleting user accounts](https://developers.arcgis.com/python/guide/accessing-and-managing-users/#deleting-user-accounts)_ section in the [Accessing and Managing Users Guide](https://developers.arcgis.com/python/guide/accessing-and-managing-users) for examples.

In [36]:
%%capture --no-stdout --no-display

new_user.delete()

True

#### Create additional users

Let's examine available _user_types_ and licenses available before we create new users. First let's examine the count of _user_types_ consumed in the organization.

##### Establish inventory of available licenses

In [37]:
utype_counts = umgr.counts('user_type')

In [38]:
utype_counts

,key,count
0,advancedUT,25
1,creatorUT,46
2,GISProfessionalAdvUT,7


In [39]:
utype_counts.rename(columns={"key":"userType"}, inplace=True)

In [40]:
utype_counts

,userType,count
0,advancedUT,25
1,creatorUT,46
2,GISProfessionalAdvUT,7


In [41]:
import pandas as pd

In [42]:
import warnings

warnings.filterwarnings("ignore", category=Warning)

utype_df = pd.DataFrame(
    data=[(ut['id'],ut['name'],ut['maxUsers']) for ut in umgr.license_types],
    columns=['userType', 'userType Name', 'Maximum']
)

In [43]:
utype_df

,userType,userType Name,Maximum
0,advancedUT,Advanced,110
1,basicUT,Basic,110
2,creatorUT,Creator,220
3,editorUT,Contributor,110
4,fieldWorkerUT,Mobile Worker,110
5,GISProfessionalAdvUT,Professional Plus,110
6,GISProfessionalBasicUT,GIS Professional Basic,0
7,GISProfessionalStdUT,Professional,110
8,IndoorsUserUT,Indoors User,110
9,insightsAnalystUT,Insights Analyst,0


In [44]:
org_utypes = pd.merge(
    left=utype_df, 
    right=utype_counts,
    how="left",
    left_on="userType",
    right_on="userType"
)

In [45]:
org_utypes.rename(columns={"count":"Consumed"}, inplace=True)
org_utypes = org_utypes.fillna({'Consumed':0})

In [46]:
org_utypes['Remaining'] = org_utypes['Maximum'] - org_utypes['Consumed']

In [47]:
org_utypes

,userType,userType Name,Maximum,Consumed,Remaining
0,advancedUT,Advanced,110,25.0,85.0
1,basicUT,Basic,110,0.0,110.0
2,creatorUT,Creator,220,46.0,174.0
3,editorUT,Contributor,110,0.0,110.0
4,fieldWorkerUT,Mobile Worker,110,0.0,110.0
5,GISProfessionalAdvUT,Professional Plus,110,7.0,103.0
6,GISProfessionalBasicUT,GIS Professional Basic,0,0.0,0.0
7,GISProfessionalStdUT,Professional,110,0.0,110.0
8,IndoorsUserUT,Indoors User,110,0.0,110.0
9,insightsAnalystUT,Insights Analyst,0,0.0,0.0


##### Create user with default role

In [48]:
new_user_contrib = gis.users.create(
    username="unique_user_X43_pmj",
    password="IL0V3MyCatB0b!Always2",
    firstname="Cynthia",
    lastname="Coleridge-Taylor",
    email="jyaist@esri.com",
    role="Data Editor",
    user_type="Contributor"
)

new_user_contrib

<User username:unique_user_X43_pmj>

In [49]:
print(f"{new_user_contrib.username:25}{new_user_contrib.role:15}{new_user_contrib.userLicenseTypeId}")

unique_user_X43_pmj      org_user       editorUT


In [50]:
%%capture --no-stdout --no-display

new_user_contrib.delete()

True

##### Create user with custom role

In [51]:
default_roles = ["org_admin", "org_publisher", "org_user"]

In [52]:
all_roles = [f"{r.name:30}{r.role_id}" for r in gis.users.roles.all()]

In [53]:
all_roles

['Viewer                        iAAAAAAAAAAAAAAA',
 'Data Editor                   iBBBBBBBBBBBBBBB',
 'Facilitator                   iCCCCCCCCCCCCCCC',
 'role_eeff8                    0mhibizUiLDIsyIY',
 'Nick Test 2                   1ZhKk7faJzCAzUkb',
 'role_84979                    3cDE784lBhmtkl7l',
 'role_d4bf8                    3lELJQcVCWIBvPL5',
 'Analyst                       53JTWAmyPz3MLgwl',
 'role_6f33c                    8MueSvFrA35xyg5p',
 'role_17509                    alKlv4u6OLgkNetV',
 'role_db009                    aojoNT0iz86h4dSY',
 'role_ea533                    BABSkINe0cyu4csO',
 'role_fda43                    bJzLJoHoSHeQIv1h',
 'AGOLImageryAnalysis           bl8ksdIBpwV4pa1N',
 'role_25fbd                    ChLk6bCpcLQX44xt',
 'AllNotebookPrivileges         cMH6sh9YL5RNFtat',
 'UC2018_GEOG471                DmZ9fQYjofiNtJWn',
 'custom_admin_role             FP3itjacJA49MYaB',
 'role_8f788                    fu6R9aBkLdOapKwb',
 'role_ddc57                   

In [54]:
new_user_tiles = gis.users.create(
    username="unique_user_C98_xlM",
    password="IL0V3MyCatB0b!Always3",
    firstname="Abraham",
    lastname="Pressich",
    email="jyaist@esri.com",
    role="tiles_publisher",
    user_type="Professional"
)

In [55]:
print(f"{new_user_tiles.username:25}{new_user_tiles.roleId:20}{new_user_tiles.userLicenseTypeId}")

unique_user_C98_xlM      gBC4ANXsKok12Lm4    GISProfessionalStdUT


### Working with Roles and User Types

<img src="./img/know_your_role.jpg" width=100 />

#### User types

- User type determines the privileges that can be granted to the member through a default or custom role
- Common Roles:
  + viewer, creator and administrator

##### Get a user and current _user_type_

In [56]:
user = gis.users.search(new_user_tiles.username)[0]

In [57]:
user.user_types()['id']

'GISProfessionalStdUT'

##### Get the ArcGIS Pro license from the organization
We can use the _check()_ method to inspect the user's current licensing for Pro.

In [58]:
%%capture --no-stdout --no-display

pro = [pl for pl in gis.admin.license.all() if "Pro" in pl.properties.listing['title']][0]
pro

<ArcGIS Pro License @ https://geosaurus.maps.arcgis.com/sharing/rest/ >

In [59]:
pro.check(user=user.username)

['desktopStdN']

##### Update the user type 
Use the _update_license_type()_ method to update the _user_type_ using the appropriate user type value.

In [60]:
user.update_license_type("GISProfessionalAdvUT")
user.user_types()['id']

'GISProfessionalAdvUT'

In [61]:
pro.check(user=user.username)

['locateXTN',
 '3DAnalystN',
 'workflowMgrN',
 'desktopAdvN',
 'geostatAnalystN',
 'dataReviewerN',
 'networkAnalystN',
 'spatialAnalystN',
 'imageAnalystN',
 'publisherN']

In [62]:
user.update_license_type("creatorUT")
user.user_types()['id']

'creatorUT'

In [63]:
pro.check(user=user.username)

['desktopBasicN']

#### Working with Roles

- A role defines the set of privileges assigned to a member

##### **Accessing Role Manager**

In [64]:
rm = gis.users.roles
rm

##### **Listing Roles**

- notice that roles can have the same name!


In [65]:
for r in rm.all():
    print(f"{r.name:30}{r.role_id:30}{r.description}")

Viewer                        iAAAAAAAAAAAAAAA              Viewer
Data Editor                   iBBBBBBBBBBBBBBB              Data Editor
Facilitator                   iCCCCCCCCCCCCCCC              Facilitator
role_eeff8                    0mhibizUiLDIsyIY              description
Nick Test 2                   1ZhKk7faJzCAzUkb              sdfsdf
role_84979                    3cDE784lBhmtkl7l              description
role_d4bf8                    3lELJQcVCWIBvPL5              description
Analyst                       53JTWAmyPz3MLgwl              A role created from the default Analyst Template for testing of it's id value
role_6f33c                    8MueSvFrA35xyg5p              description
role_17509                    alKlv4u6OLgkNetV              description
role_db009                    aojoNT0iz86h4dSY              description
role_ea533                    BABSkINe0cyu4csO              description
role_fda43                    bJzLJoHoSHeQIv1h              description
AGOLImag

##### **Check for Existence of a Role**
You can look for roles by name:

In [66]:
rm.exists('DataEditorRole')

False

##### **Create a custom role**

In [67]:
custom_role = rm.create(
    name="DataEditorRole", 
    description="Allow to modify service data", 
    privileges=[
        "features:user:edit",
        "features:user:fullEdit",
        "opendata:user:designateGroup",
        "portal:admin:viewUsers",
        "portal:user:createGroup"
    ]
)

custom_role

<Role name: DataEditorRole, description: Allow to modify service data>

In [68]:
print(f"{custom_role.name:20}{custom_role.role_id}")

DataEditorRole      CGINPJCTKwyDbwt4


##### **Create new user with defaults**

In [69]:
username = f"RUser{uuid.uuid4().hex[:4]}"
password = f"!{uuid.uuid4().hex[:8]}A"

new_crole_user = umgr.create(
    username=username, 
    password=password, 
    firstname="Ronald", 
    lastname="Jasper", 
    email='testsadf@esri.com')

new_crole_user

<User username:RUserf32d>

In [70]:
print(f"{new_crole_user.username:25}{new_crole_user.role:20}{new_crole_user.userLicenseTypeId}")

RUserf32d                org_publisher       creatorUT


##### **Update the role**

In [71]:
new_crole_user.update_role(custom_role)

True

In [72]:
new_crole_user.roleId == custom_role.role_id

True

##### **Removing the new user and role**

In [73]:
%%capture --no-stdout --no-display

new_crole_user.delete()
custom_role.delete()

True

In [74]:
%%capture --no-stdout --no-display
new_user_tiles.delete()

True